In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    break

In [2]:
# needed libraries
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from skimage.measure import label, regionprops, find_contours
import tensorflow as tf
import keras 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, UpSampling2D, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras import backend as K
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('/kaggle/input/airbus-ship-detection/train_ship_segmentations_v2.csv')

In [4]:
train

In [5]:
# decode pixels to take mask
def find_mask(encoded_pixels, size):
    my_img = []

    for i in range(0, len(encoded_pixels), 2):
        steps = encoded_pixels[i+1]
        start = encoded_pixels[i]
        start -= 1

        pos_of_pixels = [start+j for j in range(steps)]
        my_img.extend(pos_of_pixels)

    mask_img = np.zeros((size**2), dtype=np.uint8)
    mask_img[my_img] = 1
    mask = np.reshape(mask_img, (size,size)).T

    return mask


def rle_encode(mask, zeros=1e-3):
    mask = mask.T.flatten()

    mask = np.argwhere(mask > zeros)

    if mask.size > 0:
        final_ans = []
        num = 1
        first = mask[0]
        for i in range(len(mask)):
            if i+1 != len(mask):
                if mask[i]+1 == mask[i+1]:
                    num += 1
                else:
                    final_ans.extend([first[0],num])
                    first = mask[i+1]
                    num = 1
        final_ans = [str(i) for i in final_ans]

        return ' '.join(final_ans)
    else:
        return np.nan

In [6]:
np.random.seed(0)
np.random.shuffle(train.values)

In [7]:
train_without_ship = train[train['EncodedPixels'].isna()]
train_without_ship.index = [i for i in range(len(train_without_ship))]
train_with_ship = train[train['EncodedPixels'].notna()].groupby('ImageId')['EncodedPixels'].apply(lambda x: ' '.join(x)).to_frame()
train_with_ship = train_with_ship.reset_index()

In [8]:
# 4000 with shape (256,256)
n = 8000
    
# take arrays of images and coordinates for those imgs
imgs_to_classification = []
imgs_to_segmentation = []
mask_to_segmentation = []
y = []


for i in range(n):
    try:
        # read image resize it and normalize
        img = cv.imread('/kaggle/input/airbus-ship-detection/train_v2/'+train_with_ship['ImageId'][i])
        img = cv.GaussianBlur(img, (3,3), cv.BORDER_DEFAULT)
        img = cv.resize(img, (160,160))
#         img = img / 255
        img = img.astype(np.uint8)

        # decode pixels and take mask
        encoded_pixels = [int(k) for k in train_with_ship['EncodedPixels'][i].split()]
        mask = find_mask(encoded_pixels, 768)
        mask = cv.resize(mask, (160,160))
        
        imgs_to_segmentation.append(img)
        mask_to_segmentation.append(mask)

        # take 50% of images with ships and 50% without ships
        if i % 2 == 0:
            imgs_to_classification.append(img)
            y.append(np.array([1,0]))
        else:
            img = cv.imread('/kaggle/input/airbus-ship-detection/train_v2/'+train_without_ship['ImageId'][i])
            img = cv.GaussianBlur(img, (3,3), cv.BORDER_DEFAULT)
            img = cv.resize(img, (160,160))
#             img = img / 255
            img = img.astype(np.uint8)
            imgs_to_classification.append(img)
            y.append(np.array([0,1]))

    except:
#         print('Corrupted Img')
        pass

# change dtypes of our input and output data
imgs_to_classification = np.array(imgs_to_classification, dtype=np.uint8)
y = np.array(y, dtype=np.uint8)
imgs_to_segmentation = np.array(imgs_to_segmentation, dtype=np.float16)
mask_to_segmentation = np.array(mask_to_segmentation, dtype=np.float16)

In [9]:
print(imgs_to_classification.shape, imgs_to_segmentation.shape)

In [10]:
from keras import layers


# Segmentation Model
def get_model(img_size):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(1, 1, activation='sigmoid', padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model


# # Free up RAM in case the model definition cells were run multiple times
# keras.backend.clear_session()

# Build model
# with tpu_strategy.scope():
model = get_model((160,160))
# model.summary()

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

cnn3 = Sequential()
cnn3.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(160,160,3)))
cnn3.add(MaxPooling2D((2, 2)))
cnn3.add(Dropout(0.25))

cnn3.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn3.add(MaxPooling2D(pool_size=(2, 2)))
cnn3.add(Dropout(0.25))

cnn3.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn3.add(Dropout(0.4))

cnn3.add(Flatten())

cnn3.add(Dense(128, activation='relu'))
cnn3.add(Dropout(0.3))
cnn3.add(Dense(2, activation='sigmoid'))

cnn3.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
# def dice_coef(y_true, y_pred, smooth=1):
#     intersection = K.sum(y_true * y_pred, axis=[1,2,3])
#     union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
#     dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)

#     return dice


def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred))

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [13]:
from sklearn.model_selection import train_test_split

X_segm_train, X_segm_valid, y_segm_train, y_segm_valid = train_test_split(imgs_to_segmentation, mask_to_segmentation.reshape(-1,160,160,1), test_size=0.1, random_state=42)

X_class_train, X_class_valid, y_class_train, y_class_valid = train_test_split(imgs_to_classification, y, test_size=0.2, random_state=42)

In [14]:
cnn3.fit(X_class_train, y_class_train, epochs=30, batch_size=32, validation_data=(X_class_valid, y_class_valid))

In [15]:
del X_class_train
del y_class_train
del X_class_valid
del y_class_valid

In [16]:
# from tensorflow.keras.utils import Sequence
# import numpy as np   

# class DataGenerator(Sequence):
#     def __init__(self, x_set, y_set, batch_size):
#         self.x, self.y = x_set, y_set
#         self.batch_size = batch_size

#     def __len__(self):
#         return int(np.ceil(len(self.x) / float(self.batch_size)))

#     def __getitem__(self, idx):
#         batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
#         batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
#         return batch_x, batch_y

# train_gen = DataGenerator(X_segm_train, y_segm_train, 32)

In [17]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss=[dice_loss], metrics=[dice_coef])
model.fit(X_segm_train, y_segm_train, batch_size=32, epochs=100, validation_data=(X_segm_valid, y_segm_valid))

In [18]:
# imgs = np.array([cv.imread('/kaggle/input/airbus-ship-detection/train_v2/'+train_with_ship.loc[3003]['ImageId']),\
#                 cv.imread('/kaggle/input/airbus-ship-detection/train_v2/'+train_with_ship.loc[2000]['ImageId']),\
#                 cv.imread('/kaggle/input/airbus-ship-detection/train_v2/'+train_without_ship.loc[788]['ImageId']),\
#                 cv.imread('/kaggle/input/airbus-ship-detection/train_v2/'+train_with_ship.loc[555]['ImageId']),\
#                 cv.imread('/kaggle/input/airbus-ship-detection/train_v2/'+train_without_ship.loc[555]['ImageId']),\
#                 cv.imread('/kaggle/input/airbus-ship-detection/train_v2/'+train_without_ship.loc[732]['ImageId'])])

# imgs = np.array([cv.resize(i, (256,256)) for i in imgs])
# print(imgs.shape)


# final_results = []
# i = 0
# for predicted in model.predict(imgs):
#     if predicted[0] == 1:
#         final_results.append([i, model.predict(imgs[i].reshape(1, 256, 256, 3))])
        
#     i += 1


ind = 9694
img = cv.imread('/kaggle/input/airbus-ship-detection/train_v2/'+train_with_ship.loc[ind]['ImageId'])
img = cv.resize(img, (256,256))
mask = find_mask([int(i) for i in train_with_ship.loc[ind]['EncodedPixels'].split()], 768)

fig = plt.figure(figsize=(16,9))

fig.add_subplot(1,3,1)
plt.imshow(img)
fig.add_subplot(1,3,2)
plt.imshow(mask)
fig.add_subplot(1,3,3)
plt.imshow(model.predict(img.reshape(1,256,256,3)).reshape(256,256,1))